<i> Markuis Graylee adalah seorang metuber yang membuat konten mengenai produk-produk elektronik. Untuk salah satu ide kontennya, ia ingin mereview smartphone yang telah dirilis oleh beberapa perusahaan. Sebagai pendukung penelitian kontennya, ia memiliki akses pada sebuah dataset yang berisi informasi-informasi relevan yang dapat diteliti untuk menilai produk-produk yang dirilis perusahaan. Markuis juga ingin menemukan pola-pola atau hubungan yang dimiliki oleh produk-produk tersebut agar ia dapat mengetahui tren yang ada pada dunia per-smartphone-an.

Atribut: 
- battery_power
- clock_speed
- ram
- n_cores
- use_time
- px_width
- px_height
- brand
- 5g
- grade
- price 
</i>

<b>Gunakan alpha = 0.05</b>


# No. 5 Hipotesis 1 Sampel

In [1]:
import pandas as pd
import scipy.stats as stats
%run util.py

In [2]:
import scipy.stats as stats
import scipy.special as special
import math

def t_distribution(t, df):
    # distribusi t
    # f(x|df) = gamma((df+1)/2) / (sqrt(df*pi) * gamma(df/2)) * (1 + t^2/df)^-((df+1)/2)
    pembilang = special.gamma((df + 1) / 2)
    penyebut = (special.gamma(df / 2) * math.sqrt(df * math.pi))
    pengali = 1 + ((t ** 2) / df)
    pangkat = -((df + 1) / 2)

    return (((pembilang / penyebut) * (pengali ** pangkat)))

def count_p_value(t, df, tailed):
    # menghitung p-value
    if tailed == 2: # two-tailed
        return 2 * stats.t.sf(abs(t), df)
    elif tailed == 1: # one-tailed right
        return stats.t.sf(t, df)
    else:
        return stats.t.sf(-t, df)

def count_t(x_bar, mu, s, n):
    # menghitung t
    return (x_bar - mu) / (s / math.sqrt(n))

In [3]:
def changeOutliers(data, atribut):
    # Mengganti outliers dengan nilai median
    Q1 = myQ1(data, atribut)
    Q3 = myQ3(data, atribut)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    data.loc[(data[atribut] < lower_bound) | (data[atribut] > upper_bound), atribut] = data[atribut].median()
    return data

# Mengganti outliers dengan nilai median
DATA = changeOutliers(DATA, "battery_power")
DATA = changeOutliers(DATA, "clock_speed")
DATA = changeOutliers(DATA, "ram")
DATA = changeOutliers(DATA, "n_cores")
DATA = changeOutliers(DATA, "use_time")
DATA = changeOutliers(DATA, "px_width")
DATA = changeOutliers(DATA, "px_height")
DATA = changeOutliers(DATA, "5g")
DATA = changeOutliers(DATA, "price")

## 5.1 Testimoni dari pengguna banyak yang menyatakan bahwa kapasitas baterai yang digunakan kurang dari sewajarnya. Periksa apakah battery_power memiliki rata-rata di atas 1800?

Untuk memeriksa apakah rata-rata kapasitas baterai (battery_power) di atas 1800, akan digunakan uji hipotesis untuk rata-rata populasi. <br>Dalam kasus ini, akan digunakan uji t satu sampel karena kita tidak mengetahui standar deviasi dari populasi.

Langkah-langkah:
1. Hipotesis :
      $$H_0 : μ = 1800$$                 
      $$H_1 : μ > 1800$$
      μ = 1800 akan mewakili testimoni pelanggan (μ ≤ 1800)
      
2. Tingkat signifikansi : 
      $$α = 0.05$$

3. Tes statistik :
   <br>Akan digunakan uji t. Statistik uji t untuk rata-rata populasi adalah:
      $$t = (x_{bar} - μ_0) / (s / √n)$$
      dengan derajat kebebasan
      $$v = n - 1$$
      dimana:
      - xbar      : rata-rata sampel
      - μ0        : rata-rata yang diuji
      - s         : standar deviasi sampel
      - n         : ukuran sampel

    Adapun daerah kritis untuk uji hipotesis satu arah kanan dengan 
    $$α = 0.05 $$
    $$v = 1999$$
    $$v → ∞$$ 
    adalah:

    $$t_{kritis} > t_{0.05}$$
    <br>dari tabel distribusi t, diperoleh
    $$t_{kritis} > 1.645$$

4. Hitung P_Value dari X
   - Pertama hitung nilai t_{score} untuk X_{bar} dengan rumus sebelumnya
   - Cari nilai P-Value untuk P(t > t_score)
      $$P_{Value} = P(t > t_{score})$$

In [4]:
miu_0 = 1800
alpha = 0.05

# perhitungan rata-rata dan standar deviasi
my_x_bar = myMean(DATA, "battery_power")
my_s = myStdDev(DATA, "battery_power")

library_x_bar = DATA["battery_power"].mean()
library_s = DATA["battery_power"].std()

# ciritcal value
critical_value = stats.t.ppf(1 - alpha, len(DATA) - 1)  # dikurang 1 karena ppf menghitung kumulatif dari kiri, bukan kanan
                                                        # sedangkan uji yang dilakukan adalah uji one-tailed right

# perhitungan t
my_t = (count_t(my_x_bar, miu_0, my_s, len(DATA)))
library_t = (stats.ttest_1samp(DATA["battery_power"], miu_0))

# perhitungan p-value
my_p_value = count_p_value(my_t, n, 1)
library_p_value = library_t.pvalue

print("    My x_bar: ", my_x_bar.round(5))
print("    Library x_bar: ", library_x_bar.round(5))
print()
print("    My s: ", my_s.round(5))
print("    Library s: ", library_s.round(5))
print()
print("    Critical value: ", critical_value.round(5))
print()
print("    My t: ", my_t.round(5))
print("    Library t: ", library_t.statistic.round(5))
print()
print("    My P-value: ", my_p_value.round(5))
print("    Library P-value: ", library_p_value.round(5))

    My x_bar:  1921.70977
    Library x_bar:  1921.70977

    My s:  128.37519
    Library s:  128.37519

    Critical value:  1.64562

    My t:  42.39936
    Library t:  42.39936

    My P-value:  0.0
    Library P-value:  0.0


5. Diperoleh P_value untuk kedua perhitungan (fungsi buatan sendiri dan library) = 0.
    <br> Karena P_value < α (P_value < 0.05), maka hipotesis H0 ditolak dan kita menerima hipotesis H1.
    <br> Disimpulkan bahwa terdapat bukti yang signifikan bahwa rata-rata kapasitas baterai (μ) lebih besar dari 1800.

## 5.2 Standar RAM yang dimiliki oleh suatu smartphone sekarang adalah 8 GB. Periksalah apakah rata-rata ram smartphone pada dataset adalah 8 GB?

Untuk memeriksa apakah rata-rata RAM (ram) adalah 8 GB, akan digunakan uji hipotesis untuk rata-rata populasi. <br>Dalam kasus ini, akan digunakan uji t satu sampel karena kita tidak mengetahui standar deviasi dari populasi.

Langkah-langkah:
1. Hipotesis :
      $$H_0 : μ = 8$$                 
      $$H_1 : μ ≠ 8$$
      
2. Tingkat signifikansi : 
      $$α = 0.05$$

3. Tes statistik :
   <br>Akan digunakan uji t. Statistik uji t untuk rata-rata populasi adalah:
      $$t = (x_{bar} - μ_0) / (s / √n)$$
      dengan derajat kebebasan
      $$v = n - 1$$
      dimana:
      - xbar      : rata-rata sampel
      - μ0        : rata-rata yang diuji
      - s         : standar deviasi sampel
      - n         : ukuran sampel

    Adapun daerah kritis untuk uji hipotesis dua arah dengan 
    $$α = 0.05 $$
    $$v = 1999$$
    $$v → ∞$$ 
    adalah:

    $$t_{kritis} < t_{-0.025} ∪ t_{kritis} > t_{0.025}$$
    <br>dari tabel distribusi t, diperoleh
    $$t_{kritis} < -1.960 ∪ t_{kritis} > 1.960$$

4. Hitung P_Value dari X
   - Pertama hitung nilai t_score untuk X_bar dengan rumus sebelumnya
   - Cari nilai P-Value untuk P(t > t_score)
      $$P_{Value} = 2P(t > t_{score})$$

In [5]:
miu_0 = 8
alpha = 0.05

# perhitungan rata-rata dan standar deviasi
my_x_bar = myMean(DATA, "ram")
my_s = myStdDev(DATA, "ram")

library_x_bar = DATA["ram"].mean()
library_s = DATA["ram"].std()

# ciritcal value
critical_value = stats.t.ppf(1 - alpha, len(DATA) - 1)  # dikurang 1 karena ppf menghitung kumulatif dari kiri, bukan kanan
                                                        # sedangkan uji yang dilakukan adalah uji one-tailed right

# perhitungan t
my_t = (count_t(my_x_bar, miu_0, my_s, len(DATA)))
library_t = (stats.ttest_1samp(DATA["ram"], miu_0))

# perhitungan p-value
my_p_value = count_p_value(my_t, n, 2)
library_p_value = library_t.pvalue

print("    My x_bar: ", my_x_bar.round(5))
print("    Library x_bar: ", library_x_bar.round(5))
print()
print("    My s: ", my_s.round(5))
print("    Library s: ", library_s.round(5))
print()
print("    Critical value: ", critical_value.round(5))
print()
print("    My t: ", my_t.round(5))
print("    Library t: ", library_t.statistic.round(5))
print()
print("    My P-value: ", my_p_value.round(5))
print("    Library P-value: ", library_p_value.round(5))

    My x_bar:  6.04919
    Library x_bar:  6.04919

    My s:  1.95072
    Library s:  1.95072

    Critical value:  1.64562

    My t:  -44.72332
    Library t:  -44.72332

    My P-value:  0.0
    Library P-value:  0.0


5. Diperoleh P_value untuk kedua perhitungan (fungsi buatan sendiri dan library) = 0.
    <br> Karena P_value < α (P_value < 0.05), maka hipotesis H0 ditolak dan kita menerima hipotesis H1.
    <br> Disimpulkan bahwa rata-rata RAM dari smartphone yang terdapat di dalam dataset berbeda secara signifikan dari 8 GB (μ ≠ 8)

- Tentukan Hipotesis nol (H0: θ = θ0), dimana θ bisa berupa μ, σ2, p, atau data lain berdistribusi tertentu (normal, binomial, dsc.). 
- Pilih hipotesis alternatif H1 salah dari dari θ > θ0, θ < θ0, atau θ ≠ θ0. 
- Tentukan tingkat signifikan α.
- Tentukan uji statistik yang sesuai dan tentukan daerah kritis. 
- Hitung nilai uji statistik dari data sample. Hitung p-value sesuai dengan uji statistik yang digunakan. 
- Ambil keputusan dengan TOLAK H0 jika nilai uji terletak di daerah kritis atau dengan tes signifikan, TOLAK H0 jika p-value lebih kecil dibanding tingkat signifikansi α yang diinginkan 

## 5.3 Periksa apakah 250 data pertama pada dataset memiliki rata-rata kecepatan clocking (clock_speed) tidak sama dengan 1!

Untuk memeriksa apakah rata-rata kecepatan clocking (clock_speed) tidak sama dengan 1, akan digunakan uji hipotesis untuk rata-rata populasi. <br>Dalam kasus ini, akan digunakan uji t satu sampel karena kita tidak mengetahui standar deviasi dari populasi.

Langkah-langkah:
1. Hipotesis :
      $$H_0 : μ = 8$$                 
      $$H_1 : μ ≠ 8$$
      
2. Tingkat signifikansi : 
      $$α = 0.05$$

3. Tes statistik :
   <br>Akan digunakan uji t. Statistik uji t untuk rata-rata populasi adalah:
      $$t = (x_{bar} - μ_0) / (s / √n)$$
      dengan derajat kebebasan
      $$v = n - 1$$
      dimana:
      - xbar      : rata-rata sampel
      - μ0        : rata-rata yang diuji
      - s         : standar deviasi sampel
      - n         : ukuran sampel

    Adapun daerah kritis untuk uji hipotesis dua arah dengan 
    $$α = 0.05 $$
    $$v = 1999$$
    $$v → ∞$$ 
    adalah:

    $$t_{kritis} < t_{-0.025} ∪ t_{kritis} > t_{0.025}$$
    <br>dari tabel distribusi t, diperoleh
    $$t_{kritis} < -1.960 ∪ t_{kritis} > 1.960$$

4. Hitung P_Value dari X
   - Pertama hitung nilai t_score untuk X_bar dengan rumus sebelumnya
   - Cari nilai P-Value untuk P(t > t_score)
      $$P_{Value} = 2P(t > t_{score})$$

## 5.4 Periksalah apakah data smartphone dengan merek “appa” yang memiliki rata-rata waktu penggunaan (use_time) lebih dari 8.5 tidak sama dengan 35% dari data keseluruhan?